In this notebook, we focus on **data preparation**, **cleaning**, and **preprocessing** for the **UCI Adult Income Dataset**, a popular dataset often used for classification tasks predicting whether an individual earns more or less than $50,000 annually based on demographic and work-related attributes.

Good data preprocessing is crucial for reliable and interpretable results in machine learning and analytics workflows. Here, we address common data issues such as **missing values, duplicates, and inconsistent categorical labels** while creating derived features to improve downstream analysis.

## UCI Adult Income Dataset- Data Cleaning and Preprocessing

We start by importing essential Python libraries for data handling and manipulation.

In [1]:
# Import Libararies
import pandas as pd
import numpy as np
import os

- `pandas` for structured data operations.

- `numpy` for numerical operations.

- `os` for interacting with the operating system and directory structures.


## Define and Create Directory Paths

To ensure reproducibility andorganized storage, we programmatically create directories for:

- **raw data**
- **processed data**
- **results**
- **documentation**

These directories will store intermediate and final outputs for reproducibility.


In [2]:
# Get working directory
current_dir= os.getcwd()
# Go one directory up to the root directory
project_root_dir= os.path.dirname(current_dir)
project_root_dir
# Define paths to docs folder
data_dir= os.path.join(project_root_dir,'Data')
raw_dir= os.path.join(data_dir,'Raw')
processed_dir=os.path.join(data_dir,'Processed')
# Define paths to docs folder
results_dir=os.path.join(project_root_dir,'Results')
docs_dir=os.path.join(project_root_dir,'Docs')

# Create directories if they do not exist
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(docs_dir, exist_ok=True)

## Read in the data

We load the **Adult Income dataset** as a CSV file.

Key considerations here are:

- We treat `?` as missing values (`na_values = '?'`).
- We use `skipinitialspace = True` to remove extra spaces after delimeters which is common in text-based datasets.

After loading, we inspect the first few rows.


In [3]:
adult_data_filename= os.path.join(raw_dir,"adult.csv")
adult_df = pd.read_csv(adult_data_filename, header=None, na_values='?', skipinitialspace=True)
adult_df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


We also inspect the dataset's shape. We see that the data has 32,561 rows and 15 columns.

In [4]:
adult_df.shape

(32561, 15)

In addition, we check the data types using `.info`. 

In [5]:
adult_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       32561 non-null  int64 
 1   1       30725 non-null  object
 2   2       32561 non-null  int64 
 3   3       32561 non-null  object
 4   4       32561 non-null  int64 
 5   5       32561 non-null  object
 6   6       30718 non-null  object
 7   7       32561 non-null  object
 8   8       32561 non-null  object
 9   9       32561 non-null  object
 10  10      32561 non-null  int64 
 11  11      32561 non-null  int64 
 12  12      32561 non-null  int64 
 13  13      31978 non-null  object
 14  14      32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## Data Cleaning
### 1. Assign proper column names to the columns

One of the most stricking things from the above inspection is that the dataset lacks explicit column headers. We manually assign descriptive meaningful column names based on the description of the [dataset](https://archive.ics.uci.edu/dataset/2/adult). This is critical for readability and interpretability in the subsequent steps.


We inspect again to see whether they are properly assigned.


In [6]:
adult_df.columns = ['Age','Work_class','Fnlwgt', 'Education','Education_num','Marital_Status','Occupation','Relationship','Race','Sex','Capital_Gain','Capital_Loss','Hours_per_week','Native_country','Income']

### 2. Understanding the dataset

Before proceeding with the cleaning, we would like to understanding the variables deeply. This would help guide the cleaning process. The subsequent tables detail the types, meaning and values or ranges of the variables in the dataset.

**Table 1: Summary table of the variables in the dataset**

| Variable         | Type        | Description                           | Values / Range (excluding nan)         |
|:----------------|:------------|:--------------------------------------|:----------------------|
| age              | Numeric     | Age in years                          | 17 – 90                |
| fnlwgt           | Numeric     | Final sampling weight                 | ~12,285 – 1,484,705    |
| education_num    | Numeric     | Education level in years              | 1 – 16                 |
| capital_gain     | Numeric     | Capital gain amounts (Profit from selling assets above purchase price within the survey year (in USD))                  | 0 – 99,999             |
| capital_loss     | Numeric     | Capital loss amounts (Loss from selling assets below purchase price within the survey year (in USD))                 | 0 – 4,356              |
| hours_per_week   | Numeric     | Weekly work hours                     | 1 – 99                 |
| workclass        | Categorical | Type of employment                    | 8 categories           |
| education        | Categorical | Highest level of education achieved   | 16 categories          |
| marital_status   | Categorical | Marital status                        | 7 categories           |
| occupation       | Categorical | Type of job                           | 14 categories          |
| relationship     | Categorical | Relationship within household         | 6 categories           |
| race             | Categorical | Ethnic/racial group                   | 5 categories           |
| sex              | Categorical | Gender                                 | 2 categories           |
| native_country   | Categorical | Country of origin                     | 41 categories          |
| income           | Categorical | Income category (target variable)     | 2 categories: <=50K, >50K |

**Table 2: Categorical Variables Table**
| Variable        | Unique Value         | Description                                                                 |
|:----------------|:--------------------|:----------------------------------------------------------------------------|
| workclass       | Private               | Works for a private, for-profit company                                      |
|                  | Self-emp-not-inc     | Self-employed without incorporated business status                           |
|                  | Self-emp-inc         | Self-employed with an incorporated business                                  |
|                  | Federal-gov          | Employed by the federal government                                           |
|                  | State-gov            | Employed by a state government                                               |
|                  | Local-gov            | Employed by a local government                                               |
|                  | Without-pay          | Works without receiving pay (e.g. unpaid family worker)                      |
|                  | Never-worked         | Has never worked in their lifetime                                           |
| education       | Bachelors             | Bachelor's degree                                                            |
|                  | Some-college         | Some college courses completed, no degree                                    |
|                  | 11th                 | 11th grade completed                                                         |
|                  | HS-grad              | High school graduate                                                         |
|                  | Prof-school          | Professional school (e.g. law, medicine)                                     |
|                  | Assoc-acdm           | Associate degree (academic)                                                  |
|                  | Assoc-voc            | Associate degree (vocational)                                                |
|                  | 9th                  | 9th grade completed                                                          |
|                  | 7th-8th              | 7th or 8th grade completed                                                   |
|                  | 12th                 | 12th grade, no diploma                                                       |
|                  | Masters              | Master's degree                                                              |
|                  | 1st-4th              | 1st to 4th grade completed                                                   |
|                  | 10th                 | 10th grade completed                                                         |
|                  | Doctorate            | Doctoral degree                                                              |
|                  | 5th-6th              | 5th or 6th grade completed                                                   |
|                  | Preschool            | Preschool education                                                          |
| marital-status  | Married-civ-spouse    | Married, living with spouse                                                  |
|                  | Divorced             | Divorced legally                                                             |
|                  | Never-married        | Never married                                                                |
|                  | Separated            | Separated legally but not divorced                                           |
|                  | Widowed              | Spouse deceased                                                              |
|                  | Married-spouse-absent| Married, spouse not present (e.g. estrangement)                              |
|                  | Married-AF-spouse    | Married to a spouse who is a member of the Armed Forces                      |
| occupation      | Tech-support          | Technical support jobs                                                       |
|                  | Craft-repair         | Skilled manual trade and repair jobs                                         |
|                  | Other-service        | Services not classified elsewhere                                            |
|                  | Sales                | Sales-related jobs                                                           |
|                  | Exec-managerial      | Executive and managerial roles                                               |
|                  | Prof-specialty       | Professional specialty occupations (e.g. scientist, lawyer)                 |
|                  | Handlers-cleaners    | Manual labor jobs involving cleaning, handling objects                       |
|                  | Machine-op-inspct    | Machine operators, inspectors                                                |
|                  | Adm-clerical         | Administrative and clerical jobs                                             |
|                  | Farming-fishing      | Agriculture, farming, fishing occupations                                    |
|                  | Transport-moving     | Transport and moving equipment operators                                     |
|                  | Priv-house-serv      | Private household service jobs                                               |
|                  | Protective-serv      | Protective service jobs (e.g. security, law enforcement)                     |
|                  | Armed-Forces         | Military service                                                             |
| relationship    | Wife                  | Female spouse                                                                |
|                  | Own-child            | Biological or adopted child                                                  |
|                  | Husband              | Male spouse                                                                  |
|                  | Not-in-family        | Not part of a family unit (e.g. living alone)                                |
|                  | Other-relative       | Other relative in household                                                  |
|                  | Unmarried            | Single person, not married                                                   |
| race            | White                 | White                                                                        |
|                  | Asian-Pac-Islander   | Asian or Pacific Islander                                                    |
|                  | Amer-Indian-Eskimo   | American Indian or Eskimo                                                    |
|                  | Other                | Other race not listed                                                        |
|                  | Black                | Black                                                                         |
| sex             | Female                | Female                                                                        |
|                  | Male                  | Male                                                                          |
| native-country  | United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinidad-Tobago, Peru, Hong, Holland-Netherlands |                 |
| income          | <=50K                 | Income less than or equal to USD 50,000                                         |
|                  | >50K                  | Income greater than USD 50,000                                                  |


In [7]:
np.unique(adult_df.Income.to_list())

array(['<=50K', '>50K'], dtype='<U5')

In [8]:
adult_df.isnull().sum()

Age                  0
Work_class        1836
Fnlwgt               0
Education            0
Education_num        0
Marital_Status       0
Occupation        1843
Relationship         0
Race                 0
Sex                  0
Capital_Gain         0
Capital_Loss         0
Hours_per_week       0
Native_country     583
Income               0
dtype: int64

Using `.isnull().sum()`, we identify columns with missing values. They are:

- `workclass` with 1,836 missing values
- `occupation` with 1,843 missing values
- `native_country` with 583 missing values

We address these by:

- Imputing categorical missing values with `Unknown` for the columns `workclass` and `occupation`
- Imputing categorical missing values with `Other` for the column `native_country`

This has been done to preserve data consistency while acknowledging uncertainity.

We inspect one more time to ensure we don't have any missing values.

In [9]:
# deal with missing values
adult_df['Work_class'] = adult_df['Work_class'].fillna('Unkown')
adult_df['Native_country'] = adult_df['Native_country'].fillna('other')
adult_df['Occupation'] = adult_df['Occupation'].fillna('Unkown')
adult_df.isnull().sum()

Age               0
Work_class        0
Fnlwgt            0
Education         0
Education_num     0
Marital_Status    0
Occupation        0
Relationship      0
Race              0
Sex               0
Capital_Gain      0
Capital_Loss      0
Hours_per_week    0
Native_country    0
Income            0
dtype: int64

### 3. Removing Duplicates
Duplicates can distort statistical summaries and model performance. Using `.duplicated().sum()`, we count duplicate records.

In [10]:
adult_df.duplicated().sum()

24

We then inspect the duplicated records.

In [11]:
adult_df[adult_df.duplicated(keep=False)]

,Age,Work_class,Fnlwgt,Education,Education_num,Marital_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Native_country,Income
2303,90,Private,52386,Some-college,10,Never-married,Other-service,Not-in-family,Asian-Pac-Islander,Male,0,0,35,United-States,<=50K
3917,19,Private,251579,Some-college,10,Never-married,Other-service,Own-child,White,Male,0,0,14,United-States,<=50K
4325,25,Private,308144,Bachelors,13,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,Mexico,<=50K
4767,21,Private,250051,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,10,United-States,<=50K
4881,25,Private,308144,Bachelors,13,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,Mexico,<=50K
4940,38,Private,207202,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,48,United-States,>50K
5104,90,Private,52386,Some-college,10,Never-married,Other-service,Not-in-family,Asian-Pac-Islander,Male,0,0,35,United-States,<=50K
5579,27,Private,255582,HS-grad,9,Never-married,Machine-op-inspct,Not-in-family,White,Female,0,0,40,United-States,<=50K
5805,20,Private,107658,Some-college,10,Never-married,Tech-support,Not-in-family,White,Female,0,0,10,United-States,<=50K
5842,25,Private,195994,1st-4th,2,Never-married,Priv-house-serv,Not-in-family,White,Female,0,0,40,Guatemala,<=50K


Finally, we remove them with `.drop_duplicates()`.

In [12]:
adult_df=adult_df.drop_duplicates()
adult_df.dtypes == object

Age               False
Work_class         True
Fnlwgt            False
Education          True
Education_num     False
Marital_Status     True
Occupation         True
Relationship       True
Race               True
Sex                True
Capital_Gain      False
Capital_Loss      False
Hours_per_week    False
Native_country     True
Income             True
dtype: bool

We can confirm that we have no duplicates left in the dataset at this juncture.

We also inspect the current shape of the dataset and see that we have *32,537* rows and *15* columns.

### 5. Standardize Categorical Variables

**Remove any leading or trailing spaces and convert the strings to lowercase**

To prepare categorical variables for consistent processing, we first of all remove extra spaces and convert them to lowercase. This step ensures categorical variables are clean and consistently organized.


In [13]:
categorical_cols= adult_df.columns[adult_df.dtypes == object]
for col in categorical_cols:
     adult_df.loc[:, col]= adult_df[col].str.strip().str.lower()

In [14]:
adult_df

,Age,Work_class,Fnlwgt,Education,Education_num,Marital_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Native_country,Income
0,39,state-gov,77516,bachelors,13,never-married,adm-clerical,not-in-family,white,male,2174,0,40,united-states,<=50k
1,50,self-emp-not-inc,83311,bachelors,13,married-civ-spouse,exec-managerial,husband,white,male,0,0,13,united-states,<=50k
2,38,private,215646,hs-grad,9,divorced,handlers-cleaners,not-in-family,white,male,0,0,40,united-states,<=50k
3,53,private,234721,11th,7,married-civ-spouse,handlers-cleaners,husband,black,male,0,0,40,united-states,<=50k
4,28,private,338409,bachelors,13,married-civ-spouse,prof-specialty,wife,black,female,0,0,40,cuba,<=50k
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,private,257302,assoc-acdm,12,married-civ-spouse,tech-support,wife,white,female,0,0,38,united-states,<=50k
32557,40,private,154374,hs-grad,9,married-civ-spouse,machine-op-inspct,husband,white,male,0,0,40,united-states,>50k
32558,58,private,151910,hs-grad,9,widowed,adm-clerical,unmarried,white,female,0,0,40,united-states,<=50k
32559,22,private,201490,hs-grad,9,never-married,adm-clerical,own-child,white,male,0,0,20,united-states,<=50k


In [15]:
adult_df.columns

Index(['Age', 'Work_class', 'Fnlwgt', 'Education', 'Education_num',
       'Marital_Status', 'Occupation', 'Relationship', 'Race', 'Sex',
       'Capital_Gain', 'Capital_Loss', 'Hours_per_week', 'Native_country',
       'Income'],
      dtype='object')

**Re-code the `workclass` column**

We re-code the `workclass` column to broader categories like `government`, `private`, `self-employed`, etc. Table 3 shows the new encoding:

**Table 3:  Re-encoding of the `workclass` column**

| Old categories        | New Categories     |
|:-----------------|:--------------------|
|`state-gov`| `government`|
|`local-gov`| `government`|
|`federal-gov`| `government`|
|`self-emp-not-inc`| `self-employed`|
|`self-emp-inc`| `self-employed`|
|`never-worked`| `unemployed`|
|`without-pay`| `voluntary`|

In [16]:
adult_df['Work_class'].unique()

array(['state-gov', 'self-emp-not-inc', 'private', 'federal-gov',
       'local-gov', 'unkown', 'self-emp-inc', 'without-pay',
       'never-worked'], dtype=object)

In [17]:
adult_df['Work_class'] = adult_df['Work_class'].replace({
    'state-gov': 'government',
    'local-gov': 'government',
    'federal-gov': 'government',
    'self-emp-inc': 'self-employed',
    'self-emp-not-inc': 'self-employed',
    'without-pay': 'voluntary',
    'never-worked': 'unemployed'
})
adult_df['Work_class'].unique()

array(['government', 'self-employed', 'private', 'unkown', 'voluntary',
       'unemployed'], dtype=object)

**Re-code the `education` column**

We create a new column `education_level` with broader education groups. The mapping from `education` to `education_level` is as follows:


**Table 4: Mapping from `education` to `education_level`**

| Education        | Education Level     |
|:-----------------|:--------------------|
|`bachelors`       | `tertiary`          |
|`masters`         | `tertiary`          |
|`doctorate`       | `tertiary`          |
|`prof-school`     | `tertiary`          |
|`some-college`    | `some college`      |
|`assoc-acdm`      | `associate`         |
|`assoc-voc`       | `associate`         |
|`hs-grad`         | `secondary-school graduate` |
|`12th`            | `secondary`         |
|`11th`            | `secondary`         |
|`10th`            | `secondary`         |
|`9th`             | `secondary`         |
|`7th-8th`         | `primary`           |
|`5th-6th`         | `primary`           |
|`1st-4th`         | `primary`           |
|`preschool`       | `preschool`         |


In [18]:
adult_df['Education'].unique()

array(['bachelors', 'hs-grad', '11th', 'masters', '9th', 'some-college',
       'assoc-acdm', 'assoc-voc', '7th-8th', 'doctorate', 'prof-school',
       '5th-6th', '10th', '1st-4th', 'preschool', '12th'], dtype=object)

In [19]:
adult_df.loc[:,'Education_Level'] = adult_df['Education'].map({
    'bachelors': 'tertiary',
    'masters': 'tertiary',
    'doctorate': 'tertiary',
    'prof-school': 'voluntary',
    'assoc-acdm': 'associate',
    'assoc-voc': 'associate',
    'some-college': 'some-college',
    'hs-grad': 'high school graduate',
    '11th': 'secondary',
    '12th': 'secondary',
    '10th': 'secondary',
    '9th': 'secondary',
    '7th-8th': 'primary',
    '5th-6th': 'primary',
    '1st-4th': 'primary',
    'preschool':'preschool'
})


In [20]:
adult_df['Education_Level'].unique()

array(['tertiary', 'high school graduate', 'secondary', 'some-college',
       'associate', 'primary', 'voluntary', 'preschool'], dtype=object)

**Re-code the `marital_status` column**

The categories in`marital_status` are simplified into `single`, `married`, `divorced or separated` and `widowed`. See Table 5 for details.

**Table 5:  Re-encoding of the `marital_status` column**

| Old categories        | New categories     |
|:-----------------|:--------------------|
| `never-married`| `single`|
| `married-civ-spouse`| `married`|
| `married-spouse-absent`| `divorced or separated`|
| `divorced`| `divorced or separated`|
| `separated`| `divorced or separated`|
| `married-af-spouse`| `married`|


In [21]:
adult_df['Marital_Status'].unique()

array(['never-married', 'married-civ-spouse', 'divorced',
       'married-spouse-absent', 'separated', 'married-af-spouse',
       'widowed'], dtype=object)

In [22]:
adult_df.columns

Index(['Age', 'Work_class', 'Fnlwgt', 'Education', 'Education_num',
       'Marital_Status', 'Occupation', 'Relationship', 'Race', 'Sex',
       'Capital_Gain', 'Capital_Loss', 'Hours_per_week', 'Native_country',
       'Income', 'Education_Level'],
      dtype='object')

In [23]:
adult_df['Marital_Status'] = adult_df['Marital_Status'].replace({
    'never-married': 'single',
    'married-civ-spouse': 'married',
    'divorced': 'divorced',
    'married-spouse-absent': 'divorced or separated',
    'separated': 'divorced or separated',
    'married-af-spouse': 'married',
    'widowed': 'widowed'
})
adult_df['Marital_Status'].unique()

array(['single', 'married', 'divorced', 'divorced or separated',
       'widowed'], dtype=object)

**Re-code the `occupation` column**

A new column, `occupation_grouped`, is created. This new column groups the occupations into the categories `white collar`, `blue collar`, `service`, `unknown` and `military`. The exact map
ping is illustrated in Table 6.

|  Occupation       | Occupation Grouped        |
|:-----------------|:---------------|
|`adm-clerical`|`white collar`|
|`exec-managerial`| `white collar`|
|`handlers-cleaners`| `blue collar`|
|`prof-specialty`| `white collar`|
|`other-service`| `service`|
|`sales`| `white collar`|
|`craft-repair`| `blue collar`|
|`transport-moving`| `blue collar`|
|`farming-fishing`| `blue collar`|
|`machine-op-inspct`|`blue collar`|
|`tech-support`| `white collar`|
|`protective-serv`|`service`|
|`armed-forces`| `military`|
|`priv-house-serv`| `service`|
|`unknown`| `unknown`|


In [24]:
adult_df['Occupation'].unique()

array(['adm-clerical', 'exec-managerial', 'handlers-cleaners',
       'prof-specialty', 'other-service', 'sales', 'craft-repair',
       'transport-moving', 'farming-fishing', 'machine-op-inspct',
       'tech-support', 'unkown', 'protective-serv', 'armed-forces',
       'priv-house-serv'], dtype=object)

In [25]:
adult_df['Occupation_grouped'] = adult_df['Occupation'].map({
    'adm-clerical': 'white collar',
    'exec-managerial': 'white collar',
    'handlers-cleaners': 'blue collar',
    'prof-specialty': 'white collar',
    'other-service': 'service',
    'sales': 'white collar',
    'craft-repair': 'blue collar',
    'transport-moving': 'blue collar',
    'farming-fishing': 'blue collar',
    'machine-op-inspct': 'blue collar',
    'tech-support': 'white collar',
    'unkown': 'unknown',
    'protective-serv': 'service',
    'armed-forces': 'military',
    'priv-house-serv': 'service'
})


In [26]:
adult_df['Occupation_grouped'].unique()

array(['white collar', 'blue collar', 'service', 'unknown', 'military'],
      dtype=object)

**Re-code the `relationship` column**

We normalize the `race` column to indicate roles within a family or individual status.

Table 7 shows the re-encoding:

**Table 7:  Re-encoding of the `race` column**

| Old relationship        | New relationship     |
|:-----------------|:--------------------|
|`wife`| `female spouse`|
|`own-child`| `child`|
|`not-in-family`| `single`|
|`other-relative`| `extended relative`|
|`unmarried`| `single`|
|`husband`| `male spouse`|


In [27]:
adult_df['Relationship'].unique()

array(['not-in-family', 'husband', 'wife', 'own-child', 'unmarried',
       'other-relative'], dtype=object)

In [28]:
adult_df['Relationship'] = adult_df['Relationship'].replace({
    'wife': 'female spouse',
    'own-child': 'child',
    'not-in-family': 'single',
    'other-relative': 'extended relative',
    'unmarried': 'single',
    'husband': 'male spouse'
})
adult_df['Relationship'].unique()

array(['single', 'male spouse', 'female spouse', 'child',
       'extended relative'], dtype=object)

**Re-code the `race` column**

We standardize the `race` column to have more clear names. Table 8 shows the record values that were re-encoded:

**Table 8:  Re-encoding of the `race` column**

| Old categories        | New categories     |
|:-----------------|:--------------------|
|`asian-pac-islander`| `asian or pacific islander`|
|`amer-indian-eskimo`| `american indian or eskimo`|


In [29]:
adult_df['Race'].unique()

array(['white', 'black', 'asian-pac-islander', 'amer-indian-eskimo',
       'other'], dtype=object)

In [30]:
adult_df['Race'] = adult_df['Race'].replace({
    'white': 'white',
    'asian-pac-islander': 'asian or pacific islander',
    'amer-indian-eskimo': 'american indian or eskimo',
    'black': 'black',
    'other': 'other'
})
adult_df['Race'].unique()

array(['white', 'black', 'asian or pacific islander',
       'american indian or eskimo', 'other'], dtype=object)

**Re-code the `native_country` column**

We create a new colum `native_region` which maps `native_country` to geographical regions (e.g., `north america`, `asia`, etc.). The mapping is as follows:


**Table 9: Mapping from `native_country` to `native_region`**

| native_country      | regioncountry     |
|:-----------------|:--------------------|
|`united-states`| `north america`|
|`canada`| `north america`|
|`puerto-rico`| `north america`|
|`outlying-us(guam-usvi-etc)`| `north america`|
|`mexico`| `north america`|
|`cuba`| `central america`|
|`jamaica`| `central america`|
|`honduras`| `central america`|
|`dominican-republic`| `central america`|
|`el-salvador`| `central america`|
|`guatemala`| `central america`|
|`nicaragua`| `central america`|
|`trinadad&tobago`| `central america`|
|`haiti`| `central america`|
|`columbia`| `south america`|
|`ecuador`| `south america`|
|`peru`| `south america`|
|`south`| `south america`|
|`india`| `asia`|
|`china`| `asia`|
|`iran`| `asia`|
|`japan`| `asia`|
|`philippines`| `asia`|
|`cambodia`| `asia`|
|`thailand`| `asia`|
|`laos`| `asia`|
|`taiwan`| `asia`|
|`vietnam`| `asia`|
|`hong`| `asia`|
|`england`| `europe`|
|`germany`| `europe`|
|`france`| `europe`|
|`italy`| `europe`|
|`poland`| `europe`|
|`portugal`| `europe`|
|`yugoslavia`| `europe`|
|`scotland`| `europe`|
|`greece`| `europe`|
|`ireland`| `europe`|
|`hungary`| `europe`|
|`holand-netherlands`| `europe`|
|`other`| `other`


In [31]:
adult_df['Native_country'].unique()

array(['united-states', 'cuba', 'jamaica', 'india', 'other', 'mexico',
       'south', 'puerto-rico', 'honduras', 'england', 'canada', 'germany',
       'iran', 'philippines', 'italy', 'poland', 'columbia', 'cambodia',
       'thailand', 'ecuador', 'laos', 'taiwan', 'haiti', 'portugal',
       'dominican-republic', 'el-salvador', 'france', 'guatemala',
       'china', 'japan', 'yugoslavia', 'peru',
       'outlying-us(guam-usvi-etc)', 'scotland', 'trinadad&tobago',
       'greece', 'nicaragua', 'vietnam', 'hong', 'ireland', 'hungary',
       'holand-netherlands'], dtype=object)

In [32]:
adult_df['Native_place'] = adult_df['Native_country'].map({
    'united-states': 'north america',
    'cambodia': 'asia',
    'england': 'europe',
    'puerto-rico': 'north america',
    'canada': 'north america',
    'germany': 'europe',
    'outlying-us(guam-usvi-etc)': 'north america',
    'india': 'asia',
    'japan': 'asia',
    'greece': 'europe',
    'south': 'south america',
    'china': 'asia',
    'cuba': 'central america',
    'iran': 'asia',
    'honduras': 'central america',
    'philippines': 'asia',
    'italy': 'europe',
    'poland': 'europe',
    'jamaica': 'central america',
    'vietnam': 'asia',
    'mexico': 'north america',
    'portugal': 'europe',
    'ireland': 'europe',
    'france': 'europe',
    'dominican-republic': 'central america',
    'laos': 'asia',
    'ecuador': 'south america',
    'taiwan': 'asia',
    'haiti': 'central america',
    'columbia': 'south america',
    'hungary': 'europe',
    'guatemala': 'central america',
    'nicaragua': 'central america',
    'scotland': 'europe',
    'thailand': 'asia',
    'yugoslavia': 'europe',
    'el-salvador': 'central america',
    'trinadad&tobago': 'central america',
    'peru': 'south america',
    'hong': 'asia',
    'other': 'other',
    'holand-netherlands': 'europe'
})
adult_df['Native_place'].unique()

array(['north america', 'central america', 'asia', 'other',
       'south america', 'europe'], dtype=object)

In [33]:
adult_df

,Age,Work_class,Fnlwgt,Education,Education_num,Marital_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Native_country,Income,Education_Level,Occupation_grouped,Native_place
0,39,government,77516,bachelors,13,single,adm-clerical,single,white,male,2174,0,40,united-states,<=50k,tertiary,white collar,north america
1,50,self-employed,83311,bachelors,13,married,exec-managerial,male spouse,white,male,0,0,13,united-states,<=50k,tertiary,white collar,north america
2,38,private,215646,hs-grad,9,divorced,handlers-cleaners,single,white,male,0,0,40,united-states,<=50k,high school graduate,blue collar,north america
3,53,private,234721,11th,7,married,handlers-cleaners,male spouse,black,male,0,0,40,united-states,<=50k,secondary,blue collar,north america
4,28,private,338409,bachelors,13,married,prof-specialty,female spouse,black,female,0,0,40,cuba,<=50k,tertiary,white collar,central america
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,private,257302,assoc-acdm,12,married,tech-support,female spouse,white,female,0,0,38,united-states,<=50k,associate,white collar,north america
32557,40,private,154374,hs-grad,9,married,machine-op-inspct,male spouse,white,male,0,0,40,united-states,>50k,high school graduate,blue collar,north america
32558,58,private,151910,hs-grad,9,widowed,adm-clerical,single,white,female,0,0,40,united-states,<=50k,high school graduate,white collar,north america
32559,22,private,201490,hs-grad,9,single,adm-clerical,child,white,male,0,0,20,united-states,<=50k,high school graduate,white collar,north america


### 6. Create age groups based on the `age` column

Age is binned into groups such as `<18`, `18-25`, $\cdots$, `76+` to facilitate easier demographic analysis.


In [34]:
adult_df['Age'].unique()

array([39, 50, 38, 53, 28, 37, 49, 52, 31, 42, 30, 23, 32, 40, 34, 25, 43,
       54, 35, 59, 56, 19, 20, 45, 22, 48, 21, 24, 57, 44, 41, 29, 18, 47,
       46, 36, 79, 27, 67, 33, 76, 17, 55, 61, 70, 64, 71, 68, 66, 51, 58,
       26, 60, 90, 75, 65, 77, 62, 63, 80, 72, 74, 69, 73, 81, 78, 88, 82,
       83, 84, 85, 86, 87], dtype=int64)

In [35]:
# Make sure Age is numeric
adult_df['Age'] = pd.to_numeric(adult_df['Age'], errors='coerce')

# Define bins and labels
bins = [0, 18, 25, 35, 45, 60, 75, 100]
labels = ['<18', '18-25', '26-35', '36-45', '46-60', '61-75', '76+']

# Create new column for age groups
adult_df['Age_group'] = pd.cut(
    adult_df['Age'],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)


In [36]:
##pd.cut([0,1,2], bins=[0,2,4], right=False)

### 7. Drop unnecessary columns

After recoding, some columns such as `education`, `native_country` and `occupation` become redundant. We drop them to avoid multicollinearity and simplify our dataset. We notably retain the `age` column in case there is need to model it as a continuous variable.


In [37]:
adult_df.drop(column['Education', "Native_country",'Occupation', inplace=True])

In [38]:
adult_df['Age_group'].unique()

['36-45', '46-60', '26-35', '18-25', '<18', '76+', '61-75']
Categories (7, object): ['<18' < '18-25' < '26-35' < '36-45' < '46-60' < '61-75' < '76+']

## Save the Clean Dataset

Before saving the clean dataset, we re-inspect it to ensure no new issues have risen up due to re-encoding. We first of all inspect the shape of the dataset. We see that we have *32,537* rows and *16* columns. This means that there is a new column, `age_group`, added to the original dataset.


In [39]:
adult_df.shape

(32537, 19)

In [40]:
adult_df.isnull().sum()

Age                   0
Work_class            0
Fnlwgt                0
Education             0
Education_num         0
Marital_Status        0
Occupation            0
Relationship          0
Race                  0
Sex                   0
Capital_Gain          0
Capital_Loss          0
Hours_per_week        0
Native_country        0
Income                0
Education_Level       0
Occupation_grouped    0
Native_place          0
Age_group             0
dtype: int64

We confirm that there are no null values.

However, we note that there are new duplicated values given that we merged some categories in the re-encoding process. We inadvertently drop the duplicates.


In [41]:
adult_df.duplicated().sum()

1

In [42]:
adult_df=adult_df.drop_duplicates()

In [43]:
adult_df.duplicated().sum()

0

The final shape of the clean dataset is thus *32,536* rows and *19* columns.

In [44]:
adult_df.shape

(32536, 19)

Finally, we save the clean, processed dataset as a CSV file in our `processed` directory for future modelling and analysis.

In [45]:
# file_name = "Adult_Result.csv"
# file_path = os.path.join(processed_dir, file_name)
# adult_df.to_csv(file_path, index=True)